# Decision Trees


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import graphviz   # will be used to visualize the trees

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score
from sklearn import preprocessing
from sklearn.tree import export_text

In this practice session, we will consider the titanic dataset and the problem will be to predict whether the given passenger has survived the disaster or not.

In [3]:
url = "https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv"
titanic_df = pd.read_csv(url)
print(titanic_df.head())

   Survived  Pclass                                               Name  \
0         0       3                             Mr. Owen Harris Braund   
1         1       1  Mrs. John Bradley (Florence Briggs Thayer) Cum...   
2         1       3                              Miss. Laina Heikkinen   
3         1       1        Mrs. Jacques Heath (Lily May Peel) Futrelle   
4         0       3                            Mr. William Henry Allen   

      Sex   Age  Siblings/Spouses Aboard  Parents/Children Aboard     Fare  
0    male  22.0                        1                        0   7.2500  
1  female  38.0                        1                        0  71.2833  
2  female  26.0                        0                        0   7.9250  
3  female  35.0                        1                        0  53.1000  
4    male  35.0                        0                        0   8.0500  


Let's drop the column Name since it doesn't contain any signal for the classification problem.

In [4]:
# Explore and preprocess the dataset
titanic_df.drop(['Name'], axis=1, inplace=True)

In [5]:
titanic_df.head()

,Survived,Pclass,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


Let's check whether there are any missing values in the data.

In [6]:
titanic_df.isna().sum()

Survived                   0
Pclass                     0
Sex                        0
Age                        0
Siblings/Spouses Aboard    0
Parents/Children Aboard    0
Fare                       0
dtype: int64

We will need to encode the Sex columns before proceeding to the modelling phase.

In [7]:
titanic_df_ = pd.get_dummies(titanic_df, columns=['Sex'], drop_first=True)
X = titanic_df_.drop('Survived', axis=1)
y = titanic_df_['Survived']

In [8]:
X.head()

,Pclass,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Sex_male
0,3,22.0,1,0,7.2500,True
1,1,38.0,1,0,71.2833,False
2,3,26.0,0,0,7.9250,False
3,1,35.0,1,0,53.1000,False
4,3,35.0,0,0,8.0500,True


Since we do not have a separate test set, we need to construct it from the whole dataset.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=0)


In [10]:
print("Class distribution in the train set:\n", y_train.value_counts())
print("Class distribution in the test set:\n", y_test.value_counts())

Class distribution in the train set:
 Survived
0    383
1    237
Name: count, dtype: int64
Class distribution in the test set:
 Survived
0    162
1    105
Name: count, dtype: int64


The bellow cell

* creates a decision tree classifier
* builds the actual tree on the training data
* performs inference with the constructed tree on the test set




In [18]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [20]:
accuracy_train = accuracy_score(y_train, clf.predict(X_train))
accuracy_test = accuracy_score(y_test, y_pred)
print(f'Accuracy on the train set: {accuracy_train:.2f}')
print(f'Accuracy on the test set: {accuracy_test:.2f}')

Accuracy on the train set: 0.98
Accuracy on the test set: 0.77


In classification tasks, accuracy is not the only metric that is used to assess the performance of a classifier.

These metrics provide insights into the classifier's ability to make correct predictions, particularly in binary and multiclass classification scenarios. The key classification metrics include:

1. **Precision**: Precision measures the accuracy of positive predictions made by the classifier. It answers the question:

"**Of all the instances predicted as positive, how many were actually positive?**"

A higher precision indicates fewer false positives.

  **Formula**: $$\text{Precision} = \frac{TP}{TP + FP}$$

  **TP** (True Positives): The number of correctly predicted positive instances.

  **FP** (False Positives): The number of instances predicted as positive but actually negative.

2. **Recall** (Sensitivity or True Positive Rate): Recall measures the ability of the classifier to correctly identify positive instances. It answers the question:

"**Of all the actual positive instances, how many were correctly predicted as positive**?"

A higher recall indicates fewer false negatives.

  **Formula**: $$\text{Recall} = \frac{TP}{TP + FN}$$

  **TP** (True Positives): The number of correctly predicted positive instances.
  
  **FN** (False Negatives): The number of instances predicted as negative but actually positive.

3. **F1-Score**: The F1-Score is the harmonic mean of precision and recall. It provides a single metric that balances both precision and recall. It's useful when you want to find a balance between minimizing false positives and false negatives.

  **Formula**: $$\text{F1-Score} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$$

In [22]:
print("Classification Report:\n", classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.80      0.81       162
           1       0.70      0.71      0.71       105

    accuracy                           0.77       267
   macro avg       0.76      0.76      0.76       267
weighted avg       0.77      0.77      0.77       267



In [25]:
precision_score(y_true=y_test, y_pred=y_pred, pos_label=0)

0.8125

**Support** is the number of true instances for each class.

**Macro Avg** is the arithmetic mean of precision, recall, and F1-score across all classes, treating each class equally.

**Weighted Avg** is the mean of precision, recall, and F1-score across classes, weighted by the number of instances (support) in each class.

In [26]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[130  32]
 [ 30  75]]


![](https://miro.medium.com/v2/resize:fit:712/1*Z54JgbS4DUwWSknhDCvNTQ.png)

Let's also look at the decision tree itself and try to interpret some of its decisions.

In [ ]:
dot_data = export_graphviz(clf,
                           out_file=None,
                           feature_names=list(X.columns),
                           class_names=['Not Survived', 'Survived'],
                           filled=True,
                           rounded=True,
                           special_characters=True)

graph = graphviz.Source(dot_data)
graph.render("titanic_decision_tree")

## Hyper-parameter Tuning

### Cross Validation

![](https://towardsdatascience.com/wp-content/uploads/2023/12/1N45hocCMP0u4nXLe0WuSvw.png)

Let's find a good set of hyper-parameters in order to get a good classifier.

In [ ]:
from sklearn.model_selection import GridSearchCV

clf = DecisionTreeClassifier()

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=clf,
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy')

grid_search.fit(X_train, y_train)

# get the best hyperparameters and the corresponding model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# evaluate the best model on the test data
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Best Hyperparameters:", best_params)
print("Accuracy with the best Model:", accuracy)

## Feature Importances

Tree-based models also provide feature importances, which indicate how much each feature contributes (**reduces the impurity**) to the model's decision-making process. Features with higher importances are considered more influential in making predictions.

### Step 1: Compute the Impurity Reduction for a Split

Each time a feature $X_j$ is used for splitting, we compute the impurity reduction:


$$\Delta I = \text{Impurity}_{\text{parent}} - \left( w_{\text{left}} \times \text{Impurity}_{\text{left child}} + w_{\text{right}} \times \text{Impurity}_{\text{right child}} \right)$$

where:
* $\text{Impurity}_{\text{parent}}$ is the impurity of the node before splitting.
* $\text{Impurity}_{\text{left child}}$ and $\text{Impurity}_{\text{right child}}$ are the impurities of the resulting child nodes.
* $w_{\text{left}}$ and $w_{\text{right}}$ are the proportions of samples in each child node.

### Step 2: Aggregate the Reduction Over All Splits

The importance of feature $X_j$ is computed by summing the impurity reductions over all the splits where it is used:

$$\text{Importance}_j = \sum_{\text{splits using } X_j} \Delta I$$

### Step 3: Normalize the Importance Scores

To obtain relative feature importances, we normalize the scores so that they sum to 1:

$$\text{Feature Importance}_j = \frac{\sum \Delta I_j}{\sum \Delta I_{\text{all features}}}$$




In [ ]:
importances = best_model.feature_importances_

feature_importance_df = pd.DataFrame({'Feature': X.columns,
                                      'Importance': importances})

feature_importance_df = feature_importance_df.sort_values(by='Importance',
                                                          ascending=False)

print(feature_importance_df)

In [ ]:
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importances')
plt.show()

Sometimes you can get a better performing model if you eliminate the features of low importance. This is one thing that you will try in **HW2**.